<a href="https://colab.research.google.com/github/Faizmeraj25/deep_learning_assigment1/blob/main/dl_q2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np 
import pandas as pd
import math
import matplotlib.pyplot as plt
from keras.datasets import fashion_mnist
import tensorflow as tf
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
x_train = x_train / 255
x_test = x_test / 255
x_train = x_train.reshape(x_train.shape[0], -1)
x_test = x_test.reshape(x_test.shape[0], -1)


In [ ]:
print(x_test.shape)

(10000, 784)


In [ ]:
class NN: 
  # instantiate the weights and biases with random numbers. 
  def __init__(self, size): 
    self.W= [] 
    self.B = [] 
    self.preactivation = []
    self.activation = []
    for i in range(1, len(size)):
      w = np.random.rand(size[i], size[i-1]) /(np.sqrt(size[i]))
      b = np.random.rand( size[i]) 
      self.W.append(w)
      self.B.append(b)

###############################################################
  def normalize(self, x):
    for i in range(x.shape[0]):
      argmax = np.argmax(x[i])
      maxval = x[i][argmax]
      x[i] = (x[i])/(maxval)
    return x
###############################################################

  def sigmoid(self, x): 
    return 1/(1 + np.exp(-x))

###############################################################

  def sigmoid_derivative(self, x):
    sig = self.sigmoid(x)
    return sig*(1-sig)

###############################################################

  def softmax(self, x):
    for i in range(x.shape[0]):
      sum=0
      for j in range(x.shape[1]):
        sum=sum+np.exp(x[i][j])
      x[i]=np.exp(x[i])/sum
    return x

###############################################################

  def one_hot_encoded(self, y, size):
    return np.eye(size)[y]

###############################################################

  def cross_entropy(self, y_train, y_hat):
    loss=0
    for i in range (y_hat.shape[0]):
      loss+=-(np.log2(y_hat[i][y_train[i]]))
    return loss/y_hat.shape[0]

###############################################################

  def hadmard_mul(self, a, b):
    c = np.array(np.zeros((a.shape[0],a.shape[1])))
    for i in range(a.shape[0]):
      for j in range(a.shape[1]):
        c[i][j] = a[i][j] * b[i][j]
    return c

###############################################################
  def batch_converter(self, x, y, batch_size):
    no_datapoints = x.shape[0]
    no_batches = no_datapoints // batch_size
    x_batch = []
    y_batch = []
    for i in range(no_batches):
      s = i*batch_size
      e = min((i+1)*batch_size , x.shape[0])
      x1 = np.array(x[s:e])
      y1 = np.array(y[s:e])
      x_batch.append(x1)
      y_batch.append(y1)
    # jo datapoints last me bach jayenge wo yaha pe add kr rhe
    if no_batches * batch_size < x_train.shape[0]:
      x1 = np.array(x_train[no_batches* batch_size :])
      y1 = np.array(y_train[no_batches* batch_size :])
      x_batch.append(x1)
      y_batch.append(y1)
    return x_batch, y_batch


###############################################################

  def forward(self, input, size):
    # Calculating for the hiddlen layers
    for i in range(len(size)-2):
      Y = np.dot(input, self.W[i].T)   + self.B[i]
      # Y= self.normalize(Y)
      if i < len(self.preactivation):
        self.preactivation[i] = Y
      else:
        self.preactivation.append(Y)
      # Y_dash = self.normalize(Y)
      Z = self.sigmoid(Y)
      if i < len(self.activation):
        self.activation[i] = Z
      else:
        self.activation.append(Z)
      input = Z
    #Calculating for the output layer.
    i =  len(size)-2
    Y = np.dot(input, self.W[i].T) + self.B[i]
    # Y = self.normalize(Y)
    if i < len(self.preactivation):
        self.preactivation[i] = Y
    else:
      self.preactivation.append(Y)
    Z = self.softmax(Y)
    if i < len(self.activation):
        self.activation[i] = Z
    else:
        self.activation.append(Z)
    return self.preactivation, self.activation

###############################################################


  def backward(self, layers, x, y ,no_of_classes, preac, ac):
    no_layers = len(layers)
    grad_a = [] 
    grad_w = []
    grad_b = []
    grad_h = []
    y_onehot = self.one_hot_encoded(y, no_of_classes)
    grad_a.append(-(y_onehot - ac[len(ac)-1]))

    for i in range(no_layers-2, -1, -1):
      if i == 0:
        dw = (grad_a[no_layers-2-i].T @ x) #/ y.shape[0]
        db = np.sum(grad_a[no_layers-2-i],axis=0)/y.shape[0]
      else: 
        dw = (grad_a[no_layers-2-i].T @ ac[i-1])#/ y.shape[0]
        db = np.sum(grad_a[no_layers-2-i],axis=0)/ y.shape[0]
        dh_1 = grad_a[no_layers-2-i] @ self.W[i]
        sig = self.sigmoid_derivative(preac[i-1])
        da_1 = dh_1 * sig

        grad_h.append(dh_1)
        grad_a.append(da_1)
      grad_w.append(dw)
      grad_b.append(db)
    return grad_w, grad_b


###############################################################


  def gradient_descent(self, x_train, y_train, x_test, y_test, no_of_classes, layers, eta, epochs):
    l = len(layers)
    for ep in range(epochs):
      preac, ac = self.forward(x_train, layers)
      grad_w, grad_b = self.backward(layers, x_train, y_train, no_of_classes, preac, ac)

      for i in range(l-1):
        self.W[i] += -eta * grad_w[l-i-2]
        self.B[i] += -eta * grad_b[l-i-2]
      # print(ac[len(ac)-1])
      # preac, ac = self.forward(x, layers)
      loss = self.cross_entropy(y_train, ac[len(ac)-1])
      print("Iteration No : \t", ep+1, "\t Loss\t", loss)
    accur = self.test_accuracy(layers, x_test, y_test)
    print("Accuracy\t", accur, "%")


###############################################################


  def batch_grad_descent(self, x_train, y_train, x_test, y_test, no_of_classes, layers, eta, batch_size, n_iterations):
    x_batch, y_batch = self.batch_converter(x_train, y_train, batch_size)

    for i in range(n_iterations):
      for j in range(len(x_batch)):
        xb = x_batch[j]
        yb = y_batch[j]
        preac, ac = self.forward(xb, layers)
        grad_w, grad_b = self.backward(layers, xb, yb, no_of_classes,preac, ac)
        length = len(layers)
        for l in range(length-1):
          # print("shape",self.W[l].shape, grad_w[length-l-2].shape)
          self.W[l] += -eta * grad_w[length-l-2]
          self.B[l] += -eta * grad_b[length-l-2]
      loss = self.cross_entropy(yb, ac[len(ac)-1])
      print("Iteration No : \t", i+1, "\t Loss\t", loss)
    accur = self.test_accuracy(layers, x_test, y_test)
    print("Accuracy\t",accur, "%")



###############################################################


  def momentum_grad_descent(self, x_train, y_train, x_test, y_test, no_of_classes, layers, batch_size, eta, epochs, beta):
    l = len(layers)
    prev_w = [] 
    prev_b = [] 
    for i in range(l-1):
      prev_w.append(np.zeros(self.W[i].shape))
      prev_b.append(np.zeros(self.B[i].shape))

    x_batch, y_batch = self.batch_converter(x_train, y_train, batch_size)
    for ep in range(epochs):
      for j in range(len(x_batch)):
        xb = x_batch[j]
        yb = y_batch[j]
        preac, ac = self.forward(xb, layers)
        grad_w, grad_b = self.backward(layers, xb, yb, no_of_classes, preac, ac)
        for i in range(l-1):
          uw = beta*prev_w[i] + grad_w[l-i-2]
          ub = beta*prev_b[i] + grad_b[l-i-2]
          self.W[i] += -eta*uw
          self.B[i] += -eta*ub
          prev_w[i] = self.W[i]
          prev_b[i] = self.B[i]
      loss = self.cross_entropy(yb, ac[len(ac)-1])
      print("Iteration No : \t", ep+1, "\t Loss\t", loss)
    accur = self.test_accuracy(layers, x_test, y_test)
    print("Accuracy\t",accur, "%")




###############################################################


  def test_accuracy(self, layers, x, y):
    preac, ac = self.forward(x, layers)
    y_pred = ac[len(ac)-1]
    err_count = 0
    for i in range(y_pred.shape[0]):
      maxval = -(math.inf)
      maxind= -1
      for j in range(y_pred.shape[1]): 
        if maxval < y_pred[i][j]:
          maxval = y_pred[i][j]
          maxind = j
      if maxind != y[i]:
        err_count = err_count + 1
    return ((y.shape[0] - err_count)/y.shape[0])*100
      
###############################################################

#Batch Gradient Descent

In [ ]:
epoch = 10
eta = 0.001
layers = [784, 128, 10]
no_of_classes = 10
nn = NN(layers)
batch_size = 60
nn.batch_grad_descent(x_train, y_train, x_test, y_test, no_of_classes, layers, eta, batch_size, epoch)


Iteration No : 	 1 	 Loss	 3.3526490478748183
Iteration No : 	 2 	 Loss	 2.9390140445871986
Iteration No : 	 3 	 Loss	 2.4684183580400387
Iteration No : 	 4 	 Loss	 1.808689944845171
Iteration No : 	 5 	 Loss	 1.4716088237176812
Iteration No : 	 6 	 Loss	 1.325318135022857
Iteration No : 	 7 	 Loss	 1.1527154317057284
Iteration No : 	 8 	 Loss	 1.0653949780761063
Iteration No : 	 9 	 Loss	 0.980922395533707
Iteration No : 	 10 	 Loss	 0.9006248936889284
Accuracy	 80.21000000000001 %


# Vanilla Gradient Descent

In [ ]:
epoch = 20
eta = 0.0000009
layers = [784, 128, 10]
no_of_classes = 10
nn = NN(layers)
nn.gradient_descent(x_train, y_train,x_test, y_test, no_of_classes, layers, eta, epoch)
  

Iteration No : 	 1 	 Loss	 5.10578270321741
Iteration No : 	 2 	 Loss	 3.8258173576837664
Iteration No : 	 3 	 Loss	 3.4119897243670496
Iteration No : 	 4 	 Loss	 3.3393824212093173
Iteration No : 	 5 	 Loss	 3.3238051723542075
Iteration No : 	 6 	 Loss	 3.320968766345521
Iteration No : 	 7 	 Loss	 3.32058023434749
Iteration No : 	 8 	 Loss	 3.3205321963855954
Iteration No : 	 9 	 Loss	 3.3205223374482653
Iteration No : 	 10 	 Loss	 3.320516372036673
Iteration No : 	 11 	 Loss	 3.3205107899388384
Iteration No : 	 12 	 Loss	 3.3205052445359295
Iteration No : 	 13 	 Loss	 3.3204997019953018
Iteration No : 	 14 	 Loss	 3.3204941590222647
Iteration No : 	 15 	 Loss	 3.3204886152918993
Iteration No : 	 16 	 Loss	 3.320483070767847
Iteration No : 	 17 	 Loss	 3.3204775254418473
Iteration No : 	 18 	 Loss	 3.320471979307969
Iteration No : 	 19 	 Loss	 3.320466432360982
Iteration No : 	 20 	 Loss	 3.3204608845954566
Accuracy	 9.969999999999999 %


# Momentum Gradient Descent

In [ ]:
epoch = 10
eta = 0.006
beta = 0.9
layers = [784, 128, 10]
no_of_classes = 10
batch_size = 1024
nn = NN(layers)
nn.momentum_grad_descent(x_train, y_train, x_test, y_test, no_of_classes, layers, batch_size, eta, epoch, beta)

Iteration No : 	 1 	 Loss	 3.004217369017185
Iteration No : 	 2 	 Loss	 2.954323814937206
Iteration No : 	 3 	 Loss	 2.9309312597853006
Iteration No : 	 4 	 Loss	 2.8987458735893643
Iteration No : 	 5 	 Loss	 2.2986268114529755
Iteration No : 	 6 	 Loss	 1.349956004814287
Iteration No : 	 7 	 Loss	 1.0527738289226396
Iteration No : 	 8 	 Loss	 1.1987804634652246
Iteration No : 	 9 	 Loss	 1.0018224990789297
Iteration No : 	 10 	 Loss	 1.0527431490912378
Accuracy	 73.34 %
